In [1]:
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

##########################
# Phần 1: Đọc dữ liệu và xử lý cơ bản
##########################

file_path = "F:/KLTN/100_Customer/h100c101.csv"

# Đọc file CSV
df = pd.read_csv(file_path, delimiter=",", header=0)

print("Số lượng cột thực tế:", df.shape[1])
print("Dữ liệu mẫu:\n", df)
print()

# Xác định kho hàng (dòng đầu tiên sau tiêu đề)
depot = df.iloc[0]
print("Kho hàng:\n", depot)
print()

# Các khách hàng là các dòng còn lại
customers = df.iloc[1:].reset_index(drop=True)

# Chuyển đổi tọa độ thành numpy array
depot_location = np.array(depot[['x', 'y']].values)  # Tọa độ kho hàng
customer_locations = customers[['x', 'y']].values     # Tọa độ khách hàng

print("Tọa độ kho hàng:", depot_location)
print("Tọa độ khách hàng:\n", customer_locations)

# Thông tin khách hàng: demand, cửa sổ thời gian, thời gian đến (nếu có)
demands = customers['demand'].values
truck_capacity = 1300  # Tải trọng xe
time_windows = customers[['open', 'close']].values
service_time = 5
arrival_times = customers['time'].values  # nếu có

# Vận tốc xe (km/phút)
truck_speed = 50 / 60.0

print(f"Number of Customers: {len(customers)}")
print(f"Truck Capacity: {truck_capacity}")
print(f"Speed of Trucks: {truck_speed} km/m")
print(f"Arrival Times: {arrival_times}")

# Xây dựng đặc trưng cho mỗi node: [x, y, demand, open, close]
node_features_list = []

# Thông tin cho kho hàng (index 0): demand = 0, cửa sổ mở = 0, cửa sổ đóng lớn (không giới hạn)
depot_feat = [depot['x'], depot['y'], 0.0, 0.0, 1e6]
node_features_list.append(depot_feat)

# Thông tin cho khách hàng (index từ 1 trở đi)
for idx, row in customers.iterrows():
    feat = [row['x'], row['y'], row['demand'], row['open'], row['close']]
    node_features_list.append(feat)

# Chuyển thành tensor (sử dụng float)
node_features = torch.tensor(node_features_list, dtype=torch.float)

Số lượng cột thực tế: 6
Dữ liệu mẫu:
              x          y  demand  open  close        time
0    33.333333  41.666667       0     0   1236    0.000000
1    37.500000  58.333333      30   825    870    0.000000
2    35.000000  54.166667      10    15     67    0.000000
3    18.333333  62.500000      30    30     92    0.000000
4    25.000000  41.666667      10    10     73    0.000000
..         ...        ...     ...   ...    ...         ...
96   37.500000  25.000000      10   734    777  605.122934
97   20.833333  29.166667      10   912    969  629.292820
98   70.833333  20.833333      10   769    820  682.379189
99   37.500000  54.166667      20   997   1068  907.638079
100  25.000000  29.166667      10  1054   1127  911.272979

[101 rows x 6 columns]

Kho hàng:
 x           33.333333
y           41.666667
demand       0.000000
open         0.000000
close     1236.000000
time         0.000000
Name: 0, dtype: float64

Tọa độ kho hàng: [33.33333333 41.66666667]
Tọa độ khách hàng:

In [2]:
##########################
# Phần 2: Xây dựng mô hình MARDAM
##########################

class MARDAM(nn.Module):
    def __init__(self, node_input_dim, agent_state_dim, hidden_dim):
        """
        node_input_dim: số chiều đặc trưng của node (ở đây 5: [x, y, demand, open, close])
        agent_state_dim: số chiều đặc trưng trạng thái của xe (ở đây 4: [x, y, remaining_capacity, current_time])
        hidden_dim: kích thước không gian ẩn
        """
        super(MARDAM, self).__init__()
        self.hidden_dim = hidden_dim
        # Bộ nhúng cho đặc trưng node và trạng thái xe
        self.node_embed = nn.Linear(node_input_dim, hidden_dim)
        self.agent_embed = nn.Linear(agent_state_dim, hidden_dim)
        # Các lớp projection cho attention
        self.query_proj = nn.Linear(hidden_dim, hidden_dim)
        self.key_proj   = nn.Linear(hidden_dim, hidden_dim)
    
    def forward(self, agent_state, node_features, mask):
        """
        agent_state: tensor shape (batch, agent_state_dim)
        node_features: tensor shape (num_nodes, node_input_dim)
        mask: tensor shape (batch, num_nodes) với giá trị -1e9 cho các node không khả thi
        """
        # Nhúng trạng thái xe
        agent_emb = self.agent_embed(agent_state)  # (batch, hidden_dim)
        # Nhúng đặc trưng các node
        node_emb = self.node_embed(node_features)    # (num_nodes, hidden_dim)
        # Tính query và keys
        query = self.query_proj(agent_emb)            # (batch, hidden_dim)
        keys  = self.key_proj(node_emb)                # (num_nodes, hidden_dim)
        # Dot-product attention (chuẩn hóa theo sqrt(hidden_dim))
        scores = torch.matmul(query, keys.t()) / math.sqrt(self.hidden_dim)  # (batch, num_nodes)
        # Áp dụng mask
        scores = scores + mask
        # Softmax để được xác suất lựa chọn
        probs = F.softmax(scores, dim=-1)
        return probs


In [3]:
##########################
# Phần 3: Hàm mô phỏng (episode) với xe động và lưu lại log-prob cho huấn luyện
##########################

def simulate_episode(model, node_features, truck_capacity, truck_speed, service_time, penalty_new_vehicle=50.0):
    """
    Mô phỏng 1 episode định tuyến với số xe không cố định.
    Nếu xe không còn khả năng phục vụ khách nào, xe đó sẽ quay về depot.
    Nếu không xe nào có thể phục vụ khách (vẫn còn khách chưa phục vụ), ta khởi động thêm 1 xe mới.
    
    Trả về:
      - total_distance: tổng quãng đường đi của tất cả xe.
      - num_vehicles_used: số xe đã phục vụ (xe có ít nhất 1 khách ngoài depot).
      - vehicles: danh sách thông tin của từng xe (route, distance, trạng thái cuối).
      - all_log_probs: danh sách log-prob của các quyết định (cho policy gradient).
    """
    num_nodes = node_features.shape[0]
    depot_pos = node_features[0, :2].numpy()
    unvisited = set(range(1, num_nodes))  # khách hàng từ 1 trở đi

    vehicles = []  # mỗi xe là dict: {state, route, distance, log_probs, finished}
    # Khởi tạo xe đầu tiên
    vehicles.append({
         'state': np.array([depot_pos[0], depot_pos[1], truck_capacity, 0.0]),
         'route': [0],
         'distance': 0.0,
         'log_probs': [],
         'finished': False
    })
    
    while unvisited:
        progress_made = False  # kiểm tra xem có xe nào thực hiện bước di chuyển hay không
        for vehicle in vehicles:
            if vehicle['finished']:
                continue
            state = vehicle['state']
            current_pos = state[:2]
            remaining_capacity = state[2]
            current_time = state[3]
            
            # Xây mask cho các node (chỉ xét các khách hàng chưa phục vụ)
            mask_values = np.zeros(num_nodes)
            for j in range(num_nodes):
                if j == 0:  # Không cho phép chọn depot trong quá trình di chuyển
                    mask_values[j] = -1e9
                    continue
                if j not in unvisited:
                    mask_values[j] = -1e9
                    continue
                cust = node_features[j].numpy()  # [x, y, demand, open, close]
                demand = cust[2]
                open_time = cust[3]
                close_time = cust[4]
                # Kiểm tra tải trọng
                if demand > remaining_capacity:
                    mask_values[j] = -1e9
                    continue
                cust_pos = cust[:2]
                d = np.linalg.norm(current_pos - cust_pos)
                travel_time = d / truck_speed
                arrival_time = current_time + travel_time
                service_start = max(arrival_time, cust[3])
                if service_start > close_time:
                    mask_values[j] = -1e9
                    continue
            mask_tensor = torch.tensor(mask_values, dtype=torch.float).unsqueeze(0)
            agent_state_tensor = torch.tensor(state, dtype=torch.float).unsqueeze(0)
            
            # Lấy xác suất từ mô hình và sample action trực tiếp (không chuyển về numpy)
            probs = model(agent_state_tensor, node_features, mask_tensor)  # shape: (1, num_nodes)
            m = torch.distributions.Categorical(probs.squeeze(0))
            action = m.sample()  # action dưới dạng tensor
            log_prob = m.log_prob(action)
            action_int = action.item()
            
            # Cập nhật trạng thái xe với action được chọn (khách hàng)
            cust = node_features[action_int].numpy()
            cust_pos = cust[:2]
            d = np.linalg.norm(current_pos - cust_pos)
            travel_time = d / truck_speed
            arrival_time = current_time + travel_time
            service_start = max(arrival_time, cust[3])
            new_time = service_start + service_time
            new_capacity = remaining_capacity - cust[2]
            
            vehicle['distance'] += d
            vehicle['state'] = np.array([cust_pos[0], cust_pos[1], new_capacity, new_time])
            vehicle['route'].append(action_int)
            vehicle['log_probs'].append(log_prob)
            
            if action_int in unvisited:
                unvisited.remove(action_int)
            progress_made = True
            
            if not unvisited:
                break
        
        # Nếu không có xe nào di chuyển được mà vẫn còn khách chưa phục vụ, kích hoạt xe mới
        if not progress_made and unvisited:
            vehicles.append({
                 'state': np.array([depot_pos[0], depot_pos[1], truck_capacity, 0.0]),
                 'route': [0],
                 'distance': 0.0,
                 'log_probs': [],
                 'finished': False
            })
    
    # Đảm bảo tất cả xe đều quay về depot nếu chưa
    for vehicle in vehicles:
        if vehicle['route'][-1] != 0:
            current_pos = vehicle['state'][:2]
            depot_pos = node_features[0, :2].numpy()
            d = np.linalg.norm(current_pos - depot_pos)
            vehicle['distance'] += d
            travel_time = d / truck_speed
            vehicle['state'][3] += travel_time
            vehicle['route'].append(0)
            vehicle['finished'] = True
    
    total_distance = sum(v['distance'] for v in vehicles)
    # Số xe sử dụng là những xe có route gồm ít nhất 1 khách ngoài depot
    num_vehicles_used = len([v for v in vehicles if len(v['route']) > 2])
    
    # Gom tất cả log_probs cho loss
    all_log_probs = []
    for vehicle in vehicles:
         all_log_probs.extend(vehicle['log_probs'])
    
    return total_distance, num_vehicles_used, vehicles, all_log_probs


In [ ]:
##########################
# Phần 4: Huấn luyện mô hình MARDAM theo REINFORCE
##########################

# Các tham số mô hình
node_input_dim = 5     # [x, y, demand, open, close]
agent_state_dim = 4    # [x, y, remaining_capacity, current_time]
hidden_dim = 128       # kích thước không gian ẩn

model = MARDAM(node_input_dim, agent_state_dim, hidden_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_episodes = 500  # số episode huấn luyện
penalty = 50.0      # penalty cho xe bổ sung (sau xe đầu tiên)

for episode in range(num_episodes):
    optimizer.zero_grad()
    total_distance, num_vehicles_used, vehicles, log_probs = simulate_episode(
        model, node_features, truck_capacity, truck_speed, service_time, penalty_new_vehicle=penalty
    )
    # Tính cost: tổng quãng đường + penalty cho xe bổ sung
    cost = total_distance + penalty * (max(num_vehicles_used - 1, 0))
    reward = -cost  # muốn tối thiểu hóa cost nên reward âm
    
    # Tính loss theo REINFORCE
    if log_probs:
        loss = - torch.stack(log_probs).sum() * reward
    else:
        loss = torch.tensor(0.0, requires_grad=True)
    
    loss.backward()
    optimizer.step()
    
    if episode % 50 == 0:
        print(f"Episode {episode}: Loss = {loss.item():.2f}, Total Distance = {total_distance:.2f}, Vehicles = {num_vehicles_used}")
